In [ ]:
"""
Read an image & Brand Info -> convert to base64 >> simple python code

agentic starts here:
1. send to openai for image detailed description ->
2. send description to creative model for creative ideas prompts -> 
3. send image, brand info, creative ideas prompts to creative model for design ->
4. covert image back from base64 -> 
5. Compress image to < 1MB & 24 Bit depth & Resize to 1080x1920 for TIKTOK ->
6. save image & save design prompt.
"""

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field


c:\Projects\agentic-ai-design\venv\lib\site-packages\IPython\core\interactiveshell.py:3550: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## 1

In [6]:
IMAGE_DESCRIPTION_PROMPT_TEMPLATE = """ 
Describe the image in detail, focus on the main subject in the image usually in the center, 
extract all brand info like brand name and slogan if applicable. Make sure to include all details of the image.

return the description in json format as this:
{{
    image_description: description of the image
}}
"""

class GenerateImageDescription(BaseModel):
    image_description: str = Field(description="Detailed image description")

